In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # "0,1,2,3" # "0"

In [2]:
import numpy as np
import h5py
import cv2
from PIL import Image
import os, sys, gc
import argparse
import json
from glob import glob
from core.get_args import get_args
from core.produce_denoised_img import produce_denoised_img_no_crop
from core.utils import TedataLoader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import parmap
from functools import partial
from core.watershed import *
from core.watershed import watershed,watershed_per_img,watershed_original,segmentation_with_masking
from core.median_filter import apply_median_filter_cpu, apply_median_filter_gpu, apply_median_filter_gpu_simple
%load_ext autoreload
%autoreload 2

In [3]:
!ls /mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung_SNU_1536x3072/SET1/F16_10.png

ls: cannot access '/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung_SNU_1536x3072/SET1/F16_10.png': No such file or directory


In [4]:
!ls /mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung_SNU/\[SET\ 1\]/F8

10_F8.png  13_F8.png  16_F8.png  3_F8.png  6_F8.png  9_F8.png
11_F8.png  14_F8.png  1_F8.png	 4_F8.png  7_F8.png
12_F8.png  15_F8.png  2_F8.png	 5_F8.png  8_F8.png


In [5]:
glob("../weights/230207_FBI_Net*")

['../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_F08_MSE_Affine_layers_x17_filters_x64_cropsize_256.w',
 '../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_with_SET01020304_mixed_x_as_F#_y_as_F16_MSE_Affine_layers_x17_filters_x64_cropsize_256.w',
 '../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_F04_MSE_Affine_layers_x17_filters_x64_cropsize_256.w',
 '../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_F16_MSE_Affine_layers_x17_filters_x64_cropsize_256.w',
 '../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_F02_MSE_Affine_layers_x17_filters_x64_cropsize_256.w']

In [6]:
def get_CD(img,img_info, img_hole_info, pad, print_img=False):
    img = np.squeeze(img.copy())
    CD_info = {'min_CD' : [],'max_CD' : [], 'avg_min_CD': None , 'avg_max_CD' : None }

    for i,j in img_hole_info:
        # crop image patch
        pos = {'r_start' : i-pad, 'r_end' : i+pad, 'c_start' : j-pad, 'c_end' : j+pad}
        need_arrange = []
        for key,item in pos.items():
            if item < 0:
                pos[key] = 0
                need_arrange.append(key)
        for key in need_arrange:
            correspond_key = f"{key[0]}_end"
            pos[correspond_key] = pad*2
        img_patch = img[pos['r_start']:pos['r_end'],pos['c_start']:pos['c_end']].copy()
        # print(img_patch.shape)
        if img_patch.shape != (pad*2,pad*2):
            continue

        # leave only circle imagein patch 
        circle_val = img[i,j]
        img_patch[img_patch != circle_val] = 0

        (cX,cY), circle_with_marker = find_center(img_patch)

        if print_img is True:
            print("center : ",cX,cY)
            plt.title(f"pos : {i},{j}")
            plt.imshow(img_patch)
            plt.scatter(cX,cY,color='r',marker='.')
            plt.pause(0.01)
        search_angle = list(range(0,180))# (map(lambda x : int(x),range(0,180)))

        # find min/max CD
        # partial_func = partial(measure_CD,img=img_patch,cX=cX,cY=cY,debug=False)
        # measured_CD = parmap.map(partial_func,range(0,180), pm_processes=12)
        measured_CD = []
        for angle in range(180):
            measured_CD.append(measure_CD(angle,img_patch,cX,cY,debug=False))
        # print(measured_CD)
        if None in measured_CD:
            print(f"{img_info['set_num']}, {img_info['f_num']} {img_info['idx']} {i},{j} hole cannot get CD measurement due to None")
            continue
        min_CD,max_CD = min(measured_CD),max(measured_CD)
        CD_info['min_CD'].append(min_CD)
        CD_info['max_CD'].append(max_CD)
    
    CD_info['avg_min_CD'] = np.mean(CD_info['min_CD'])
    CD_info['avg_max_CD'] = np.mean(CD_info['max_CD'])
    return CD_info

In [7]:
def return_img(img_dict): 
    img_info = {}
    for set_num in img_dict.keys():
        for f_num in img_dict[set_num].keys():
            for idx,img in enumerate(img_dict[set_num][f_num]):
                img_info = {'set_num' : set_num, 'f_num' : f_num, 'idx' : idx}
                yield img_info, img
def denoise_and_segment_img_uint8(model, img,print_img=False):
    if img.shape != (1,1,1474,3010):
        while len(img.shape) < 4:
            img = np.expand_dims(img,axis=0)
        if len(img.shape) > 4:
            img = np.squeeze(img)
            img = np.expand_dims(img,axis=0)
            img = np.expand_dims(img,axis=0) # (1,1,1474,3010)
    # print(img.shape)
    if type(model) is type(apply_median_filter_gpu_simple) :
        denoised_img = model(img)
    else :
        denoised_img = model.eval(img)
    denoised_img_uint8 = (denoised_img[0][0]*255).astype('uint8')
    segmentataion_img_uint8 = segmentation_with_masking(denoised_img_uint8)
    if print_img is True:
        plt.imshow(segmentataion_img_uint8)
        plt.pause(0.01)
    segmentataion_img_uint8 = np.expand_dims(segmentataion_img_uint8, axis=0)
    # print(segmentataion_img_uint8.shape)
    return segmentataion_img_uint8 # (1,1474,3010)

In [8]:
def CD_process(model,img,img_info, img_hole_info,pad):
    # print(img_info)
    segmentataion_img_uint8 = denoise_and_segment_img_uint8(model,img,print_img=False)
    CD_info = get_CD(segmentataion_img_uint8,img_info, img_hole_info, pad,False)
    print(f"{img_info['set_num']}_{img_info['f_num']}_{img_info['idx']:02d}th image - Min/Max (len : {len(CD_info['min_CD'])}) CD : \
    {CD_info['avg_min_CD']:.4f} ~ {CD_info['avg_max_CD']:.4f}")
    return CD_info

In [9]:
def denoise_and_median(img):
    denoised_img = model.eval(img)
    denoised_img = apply_median_filter_gpu_simple(denoised_img)
    return denoised_img

In [10]:
pad = 150
noisy_f_num_list = ['F01', 'F02', 'F04', 'F08']
CD_dict ={}
target = 'F16_v2'
with open(f'./hole_info_{target}.txt', 'r') as f:
    hole_info = json.load(f)


In [11]:
## Denoising argument
args = get_args()
args.pge_weight_dir = None #pge_weight_dir
args.loss_function = "MSE_Affine"
args.noise_type = 'Poisson-Gaussian'
args.model_type = 'FBI_Net'
# args.set_num = '1'
args.data_name = 'Samsung'
args.data_type = 'Grayscale'
args.lr = 0.001
args.num_layers = 17
args.num_filters = 64
# args.crop_size = 200
args.debug = False
# args


img_dict = np.load("./intermediate_result/full_img_dict.npy",allow_pickle=True).item()
# fbi_weight_dir = glob("../weights/230207_FBI_Net*")[0]# '../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_F08_MSE_Affine_layers_x17_filters_x64_cropsize_256.w'
fbi_weight_dir = f'../weights/230207_FBI_Net_Grayscale_Samsung_SET050607080910_mixed_x_as_F#_y_as_{target.split("_")[0]}_MSE_Affine_layers_x17_filters_x64_cropsize_256.w'

model = produce_denoised_img_no_crop(_pge_weight_dir=None,_fbi_weight_dir = fbi_weight_dir,_args = args)

## Denoise with FBI-denoiser

In [12]:

for img_info, img in return_img(img_dict):
    if img_info['f_num'] not in noisy_f_num_list :
        continue
    print(img_info, img.shape) # (1,x,y)
    set_num, f_num, idx = img_info["set_num"], img_info["f_num"], img_info["idx"]

    key = f'{img_info["set_num"]}_{img_info["f_num"]}_{img_info["idx"]:02d}'
    CD_dict[key] = {"min_CD" : [], "max_CD" : []}
    img_hole_info = hole_info[set_num][f_num][idx]
    print(f"===== {target} CD measure =====")
    # CD_process(model, img, img_info, img_hole_info, pad)
    CD_process(model, img, img_info, img_hole_info, pad)
    print("===== median filter CD measure ===== ")
    CD_process(apply_median_filter_gpu_simple, img, img_info, img_hole_info, pad)
    print("")
    
    

{'set_num': 'SET01', 'f_num': 'F08', 'idx': 0} (1, 1474, 3010)
===== F16_v2 CD measure =====
SET01_F08_00th image - Min/Max (len : 123) CD :     145.3764 ~ 156.4038
===== median filter CD measure ===== 
SET01_F08_00th image - Min/Max (len : 123) CD :     139.2599 ~ 152.8002

{'set_num': 'SET01', 'f_num': 'F08', 'idx': 1} (1, 1474, 3010)
===== F16_v2 CD measure =====
SET01_F08_01th image - Min/Max (len : 124) CD :     144.9498 ~ 156.2309
===== median filter CD measure ===== 
SET01_F08_01th image - Min/Max (len : 124) CD :     135.1121 ~ 148.0945

{'set_num': 'SET01', 'f_num': 'F08', 'idx': 2} (1, 1474, 3010)
===== F16_v2 CD measure =====
SET01_F08_02th image - Min/Max (len : 123) CD :     146.3087 ~ 156.9917
===== median filter CD measure ===== 
SET01_F08_02th image - Min/Max (len : 123) CD :     140.0752 ~ 152.5241

{'set_num': 'SET01', 'f_num': 'F08', 'idx': 3} (1, 1474, 3010)
===== F16_v2 CD measure =====
SET01_F08_03th image - Min/Max (len : 123) CD :     145.9204 ~ 156.7972
===== m

KeyboardInterrupt: 

In [ ]:
print("=========================")

## Denoise with FBI-denoiser and apply median

In [ ]:
target = "F16_v2"
success_key = []
collect_metric_list = ["avg_min_CD", "avg_max_CD"]
method_list = [target, f"{target}_median", 'median']
method_idx = 0
def get_metric(CD_dict,local_CD_info,collect_metric_list, method_list, method_idx):
    for metric in collect_metric_list:
        CD_dict[key][f"{method_list[method_idx]}_{metric}"] = local_CD_info[metric]
    method_idx+=1
    return method_idx, CD_dict
for img_info, img in return_img(img_dict):
    if img_info['f_num'] not in noisy_f_num_list :
        continue
    print(img_info, img.shape) # (1,x,y)
    # continue
    set_num, f_num, idx = img_info["set_num"], img_info["f_num"], img_info["idx"]
    key = f'{img_info["set_num"]}_{img_info["f_num"]}_{img_info["idx"]:02d}'
    CD_dict[key] = {}
    
    img_hole_info = hole_info[set_num][f_num][idx]
    print(f"===== {target}  CD measure =====")
    CD_info_ours1 = CD_process(model, img, img_info, img_hole_info, pad)
    method_idx, CD_dict = get_metric(CD_dict, CD_info_ours1, collect_metric_list, method_list, method_idx)
    print(f"===== {target} & median CD measure =====")
    CD_info_ours2 = CD_process(denoise_and_median, img, img_info, img_hole_info, pad)
    method_idx, CD_dict = get_metric(CD_dict, CD_info_ours2, collect_metric_list, method_list, method_idx)
    if target == 'F16_v2':
        print("===== median filter CD measure ===== ")
        CD_info_median = CD_process(apply_median_filter_gpu_simple, img, img_info, img_hole_info, pad)
        method_idx, CD_dict = get_metric(CD_dict, CD_info_median, collect_metric_list, method_list, method_idx)
        
        if abs(CD_info_ours2['avg_min_CD'] - CD_info_median['avg_min_CD']) <=1 and abs(CD_info_ours2['avg_max_CD'] - CD_info_median['avg_max_CD']) <=1:
            success_key.append(key)
    print("")
    break
if target == 'F16_v2':
    np.save(f"sucess_key_{target}.npy",success_key)
    print(success_key)

CD_dict = pd.DataFrame(CD_dict)
method_str = "_".join(method_list)
CD_dict.to_csv(f"CD_dict_{method_str}.csv")

In [3]:
a = {0 : {0 : 1}}

In [8]:
a[0][3] = 2
a[0].keys()

dict_keys([0, 1, 3])